In [1]:
import urllib.request

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
urllib.request.urlretrieve(url, "spacex_launch_dash.csv")

('spacex_launch_dash.csv', <http.client.HTTPMessage at 0x227a30d1850>)

In [2]:
# exmaine url in a pandas dataframe
import pandas as pd
df = pd.read_csv("spacex_launch_dash.csv")
print(df.head(5))

   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  


In [3]:
# download a skeleton Dash app to be completed in this lab
skeleton_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"
urllib.request.urlretrieve(skeleton_url, "spacex-dash-app.py")

('spacex-dash-app.py', <http.client.HTTPMessage at 0x227a30d1190>)

In [4]:
with open('spacex-dash-app.py', 'r') as file:
	print(file.read())

# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),

                               

In [5]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the flight data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                               
# TASK 1: Add a dropdown list to enable Launch Site selection; # The default select value is for ALL sites
# dcc.Dropdown(id='site-dropdown',...)
dcc.Dropdown(id='site-dropdown',
                                 options=[
                                     {'label': 'All Sites', 'value': 'ALL'},
                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                 ],
                                 value='ALL',
                                 placeholder="Select a Launch Site",
                                 searchable=True
                                 ),

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# Note: If a specific launch site is selected from the site drop-down, then show the Success vs. Failed counts for the selected site
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
html.Div(dcc.Graph(id='success-pie-chart')),
html.Br(),


# TASK 3: Add a slider to select payload range
# The payload range is between 0 and 10000
dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                value=[min_payload, max_payload],
                marks={0: '0',
                      2500: '2500',
                      5000: '5000',
                      7500: '7500',
                      10000: '10000'}),

                             


                                html.P("Payload range (Kg):"),


                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])





# Run the app
if __name__ == '__main__':
    app.run()# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, 
                     values='class',
                     names='Launch Site',
                     title='Total Success Launches By Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df,
                     names='class',
                     title=f'Success vs Failed Launches for {entered_site}')
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)
def get_scatter_chart(entered_site, payload):
    low, high = payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Correlation between Payload and Success for all Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Correlation between Payload and Success for {entered_site}')
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server()


In [8]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

# Import required libraries
import plotly.express as px

# Read the flight data into pandas dataframe
# Note: Assuming spacex_df is already loaded from previous cells
# spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app_new = dash.Dash(__name__) # Create a new app instance to avoid conflicts if the previous cell was run

# Create an app layout
app_new.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard (Counts)',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # Dropdown for Launch Site selection
                                dcc.Dropdown(id='site-dropdown-counts',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site",
                                             searchable=True
                                             ),
                                html.Br(),

                                # Pie chart to show successful launch counts
                                html.Div(dcc.Graph(id='success-pie-chart-counts')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Slider to select payload range
                                dcc.RangeSlider(id='payload-slider-counts',
                                                min=0, max=10000, step=1000,
                                                value=[min_payload, max_payload],
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'}
                                                ),
                                html.Br(),

                                # Scatter chart to show correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart-counts')),
                                ])

# Callback function for `site-dropdown` as input, `success-pie-chart` as output (showing counts)
@app_new.callback(
    Output(component_id='success-pie-chart-counts', component_property='figure'),
    Input(component_id='site-dropdown-counts', component_property='value')
)
def get_pie_chart_counts(entered_site):
    if entered_site == 'ALL':
        # Filter for successful launches only
        successful_launches = spacex_df[spacex_df['class'] == 1]
        # Use Plotly Express to create a pie chart of successful launches by site
        # `names` determines the categories, px.pie automatically counts occurrences
        fig = px.pie(successful_launches,
                     names='Launch Site',
                     title='Total Successful Launches Count By Site')
        # Update hover information and text on slices to show counts explicitly
        fig.update_traces(textinfo='value', hoverinfo='label+percent+value')
    else:
        # Filter dataframe for the selected launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].copy() # Use .copy() to avoid warnings
        # Map class values to 'Success'/'Failed' for clarity in the chart
        filtered_df['outcome'] = filtered_df['class'].map({1: 'Success', 0: 'Failed'})
        # Create pie chart showing success vs failure counts for the specific site
        # `names` determines the categories, px.pie automatically counts occurrences
        # Add color mapping for Success (green) and Failed (red)
        fig = px.pie(filtered_df,
                     names='outcome',
                     title=f'Total Success vs Failed Launches Count for site {entered_site}',
                     color='outcome', # Specify the column to base color on
                     color_discrete_map={'Success':'green', 'Failed':'red'}) # Map values to colors
        # Update hover information and text on slices to show counts explicitly
        fig.update_traces(textinfo='value', hoverinfo='label+percent+value')
    return fig


# Callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app_new.callback(
    Output(component_id='success-payload-scatter-chart-counts', component_property='figure'),
    [Input(component_id='site-dropdown-counts', component_property='value'),
     Input(component_id='payload-slider-counts', component_property='value')]
)
def get_scatter_chart_counts(entered_site, payload_range):
    low, high = payload_range
    # Filter based on payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    # Create a fixed color map with manually chosen distinguishable colors
    color_discrete_map = {
        'v1.0': '#e41a1c',    # Red
        'v1.1': '#377eb8',    # Blue
        'FT': '#4daf4a',      # Green
        'B4': '#ff7f00',      # Orange
        'B5': '#9932cc'       # Purple
    }

    if entered_site == 'ALL':
        # Render scatter plot for all sites
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         color_discrete_map=color_discrete_map,
                         title='Correlation between Payload and Success for all Sites')
    else:
        # Filter further based on selected site
        site_filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        # Render scatter plot for the specific site
        fig = px.scatter(site_filtered_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color='Booster Version Category',
                         color_discrete_map=color_discrete_map,
                         title=f'Correlation between Payload and Success for site {entered_site}')
    
    # Customize markers for better visibility
    fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
    
    # Update y-axis to show clear labels for success/failure
    fig.update_yaxes(tickvals=[0, 1], ticktext=['Failed', 'Success'])
    
    return fig

# Run the app
if __name__ == '__main__':
    # Use a different port if the previous app might still be running
    app_new.run_server(port=8051)